# 댓글 데이터 감성분석
- Aurora3 기반 https://github.com/gyubok-lee/Aurora3
- Google colaboratory 이용시 drive mount & cd 설정 필요


In [ ]:
"""
cd /content/drive/MyDrive/경로설정
"""

In [ ]:
# konply 설치
!pip install konlpy

In [ ]:
# module & package 가져오기

import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

# 작업순서
- input data 파일 형식 바꿔주기
- test, train data 불러오기
- 데이터 전처리 작업
- 단어별로 감성점수 측정, 추정
- 댓글 문장 별 감성분석
- score 측점 결과 저장

In [ ]:
"""
- 파일 변환 -

df = pd.read_excel('파일경로/변환 전 파일이름.xlsx')
print(df)
df.to_csv('변환 후 새로 생기는 파일이름.txt',index=True)
"""

In [ ]:
# 영화제목 적어주기

movie_name = "#살아있다"

In [ ]:
df = pd.read_excel(f'data_xlsx/youtube_comments_{movie_name}.xlsx')
print(df)
df.to_csv(f'data_txt/youtube_comments_{movie_name}.txt',index=True, sep='\t')

                                                comment  ...      videoId
0                                          갓직히 라면은 진순이지  ...  wm06nhjoAQ8
1                                           나만 재밌게 봤나.?  ...  wm06nhjoAQ8
2     하..반도보고도 허무했는데..이것도 그런가요. <a href="http://www....  ...  wm06nhjoAQ8
3       그런데 제가보기엔 괜찮은게 생존의 생각을 못하고 앞에보이는거만 보는이미지 괜찮았습니다  ...  wm06nhjoAQ8
4                                               ㅈㄴ불편하넹?  ...  wm06nhjoAQ8
...                                                 ...  ...          ...
4840                                    나만 이노래가 ㅗ름 끼친가?  ...  YzS9h3T2bXY
4841  이거 들으면서 학원갈 준비하면 완전 좀비랑 <br>싸우러 가려고 준비하는 기분..<...  ...  YzS9h3T2bXY
4842                                           공부랑 싸우다니  ...  YzS9h3T2bXY
4843                                               😆😆😆😆  ...  YzS9h3T2bXY
4844                                             인정요ㅋㅋㅋ  ...  YzS9h3T2bXY

[4845 rows x 5 columns]


In [ ]:
# 입력 데이터프레임은 다음과 같은 양식으로 통일할 것
df = pd.read_table(f'data_txt/youtube_comments_{movie_name}.txt')
#영화별로 총 댓글 수 다름 -> 자동화
df = df.iloc[:,:] 
# 기존 row name을 data에 맞게 변경
df = df[['videoId','comment']]
df = df.rename(columns={"videoId": "id", "comment": "sentence"})
df

,id,sentence
0,wm06nhjoAQ8,갓직히 라면은 진순이지
1,wm06nhjoAQ8,나만 재밌게 봤나.?
2,wm06nhjoAQ8,"하..반도보고도 허무했는데..이것도 그런가요. <a href=""http://www...."
3,wm06nhjoAQ8,그런데 제가보기엔 괜찮은게 생존의 생각을 못하고 앞에보이는거만 보는이미지 괜찮았습니다
4,wm06nhjoAQ8,ㅈㄴ불편하넹?
...,...,...
4840,YzS9h3T2bXY,나만 이노래가 ㅗ름 끼친가?
4841,YzS9h3T2bXY,이거 들으면서 학원갈 준비하면 완전 좀비랑 <br>싸우러 가려고 준비하는 기분..<...
4842,YzS9h3T2bXY,공부랑 싸우다니
4843,YzS9h3T2bXY,😆😆😆😆


In [ ]:
# 데이터 전처리 및 토큰화 작업

okt = konlpy.tag.Okt()

def text_preprocess(x):
    text=[]
    """
    <br>, <a href~ > 등의 태그 제거 작업 추가 필요
    """
    x = re.sub("<.+?>","",x)
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [ ]:
# 잘 되었는지 확인

tqdm.pandas()
df['comment_cut'] = df['sentence'].apply(lambda x : text_preprocess(x))
df['comment_cut'] = df['comment_cut'].progress_apply(lambda x: tokenize(x))
df

100%|██████████| 4845/4845 [00:21<00:00, 227.86it/s]


,id,sentence,comment_cut
0,wm06nhjoAQ8,갓직히 라면은 진순이지,"[갓, 직히, 라면, 은, 진, 순이, 지]"
1,wm06nhjoAQ8,나만 재밌게 봤나.?,"[나, 만, 재밌게, 봤나]"
2,wm06nhjoAQ8,"하..반도보고도 허무했는데..이것도 그런가요. <a href=""http://www....","[하반, 도보, 고도, 허무했는데이것도, 그런, 가요, 살아만, 있다, 인가요, 오..."
3,wm06nhjoAQ8,그런데 제가보기엔 괜찮은게 생존의 생각을 못하고 앞에보이는거만 보는이미지 괜찮았습니다,"[그런데, 제, 가보기엔, 괜찮, 은, 게, 생존, 의, 생각, 을, 못, 하고, ..."
4,wm06nhjoAQ8,ㅈㄴ불편하넹?,"[불편하, 넹]"
...,...,...,...
4840,YzS9h3T2bXY,나만 이노래가 ㅗ름 끼친가?,"[나, 만, 이, 노래, 가, 름, 끼친가]"
4841,YzS9h3T2bXY,이거 들으면서 학원갈 준비하면 완전 좀비랑 <br>싸우러 가려고 준비하는 기분..<...,"[이, 거, 들으면서, 학원, 갈, 준비, 하면, 완전, 좀비, 랑, 싸우러, 가려..."
4842,YzS9h3T2bXY,공부랑 싸우다니,"[공부, 랑, 싸우다니]"
4843,YzS9h3T2bXY,😆😆😆😆,[]


In [ ]:
sent_dic = pd.read_csv('SentiWord_Dict.txt',sep = '\t',header=None)
sent_dic.iloc[14850,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

In [ ]:
# class 선언으로 한번에 작업 처리

class Aurora3:
    
    def __init__(self, df,sent_dic):
        self.df = df
        self.okt = konlpy.tag.Okt()
        self.sent_dic = sent_dic
        
    def get_df(self):# 최종 결과 반환
        #print("문장 토큰화 중입니다...")
        #self.tokenizer_run()
        
        print("감성사전 업데이트 중입니다...")
        self.expand_sent_dic()
        
        print("문장 감성분석 중입니다....")
        self.sent_analyze()
        return self.df
    """    
    def tokenizer_run(self): # 텍스트 전처리 & 토큰화
        tqdm.pandas()
        
        def text_preprocess(x): 
            text=[]
            
            x1 = re.sub("<.+?>","",x)
            a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x1)
            for j in a.split():
                text.append(j)
            return ' '.join(text)

        def tokenize(x):
            text = []
            tokens = self.okt.pos(x)
            for token in tokens :
                if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
                    text.append(token[0])
            return text

        self.df['comment_cut'] = self.df['sentence'].apply(lambda x : text_preprocess(x))
        self.df['comment_cut'] = self.df['comment_cut'].progress_apply(lambda x: tokenize(x))
    """
    def expand_sent_dic(self):
        sent_dic = self.sent_dic
        
        def make_sent_dict(x) :
            pos=[]
            neg=[]
            tmp={}

            for sentence in tqdm(x):
                for word in sentence :
                    target = sent_dic[sent_dic[0]==word]
                    if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                        score = float(target[1])
                        if score > 0:
                            pos.append(word)
                        elif score < 0:
                            neg.append(word)                
                    tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
            pos = list(set(pos))
            neg = list(set(neg))

            for sentence in tqdm(x):
                for word in sentence :
                    tmp[word]['W'] += 1 # 빈도 수
                    for po in pos :
                        if po in sentence:
                            tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                            break
                    for ne in neg:
                        if ne in sentence:
                            tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                            break
            return pos, neg, pd.DataFrame(tmp)
        
        def make_score_dict(d,p,n):
            N=sum(d.iloc[0,::])
            pos_cnt=sum(d.loc[::,p].iloc[0,::])
            neg_cnt=sum(d.loc[::,n].iloc[0,::])

            trans =d.T
            trans['neg_cnt']=neg_cnt
            trans['pos_cnt']=pos_cnt
            trans['N']=N

            trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
            trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
            trans['SO_MI']=trans['MI_P'] - trans['MI_N']

            trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            trans = trans.sort_values(by=['SO_MI'],ascending=False)
            return trans
        
        def update_dict(d):
            add_Dic = {0:[],1:[]}
            for i in d.T.items():
                if i[0] not in list(sent_dic[0]):
                    if len(i[0]) > 1:
                        add_Dic[0].append(i[0])
                        add_Dic[1].append(i[1]['SO_MI'])

            add_Dic=pd.DataFrame(add_Dic)
            Sentiment=pd.merge(sent_dic,add_Dic,'outer')
            return Sentiment
        
        self.pos, self.neg, self.new_dict = make_sent_dict(self.df['comment_cut'].values)
        
        self.t_dict = make_score_dict(self.new_dict,self.pos,self.neg)
        self.t_dict['SO_MI'] = scaler.fit_transform(self.t_dict['SO_MI'].values.reshape(-1,1))
       
        self.add_dict =update_dict(self.t_dict)
    
    def sent_analyze(self): # 데이터 감성분석
        tqdm.pandas()
        
        def get_cnt(x):
            cnt = 0
            for word in list(set(x)):
                target = self.add_dict[self.add_dict[0]==word]
                if len(target)==1:
                    cnt += float(target[1])
            return cnt

        def get_ratio(x): # log로 score 정규화
            score = x['score']
            length = np.log10(len(x['comment_cut']))+1
            try:
                ratio= round(score/length,2)
            except:
                ratio = 0
            return ratio
        
        tqdm.pandas()
        self.df['score']= self.df['comment_cut'].progress_apply(lambda x : get_cnt(x))
        self.df['ratio'] = self.df.apply(lambda x: get_ratio(x), axis = 1)

In [ ]:
# 감성분석 점수 도출

test = Aurora3(df,sent_dic)
res = test.get_df()

In [ ]:
# score 결과 파일로 저장

score_res = pd.DataFrame(res)
score_res.to_csv(f"/content/drive/MyDrive/파일경로/score_youtube_comments_{movie_name}.csv", header=True, index=True)